# Anthropic Provider Guide 🤖

This notebook demonstrates how to use Anthropic's Claude models through different hosting options:
- **Direct Anthropic**: Using Anthropic's native API
- **Vertex AI**: Using Claude through Google Cloud's Vertex AI platform

## Overview

Anthropic offers Claude models through multiple hosting options, each with their own advantages:

### Direct Anthropic Hosting
- **Direct API access** to Anthropic's servers
- **Latest models** available first
- **Simple authentication** with API key
- **Global availability** with multiple regions

### Vertex AI Hosting
- **Enterprise features** through Google Cloud
- **VPC integration** for secure deployments
- **Regional compliance** options
- **Unified billing** with other Google Cloud services

## Setup

First, let's load environment variables for authentication:

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## Direct Anthropic Hosting

### Authentication
The Anthropic client automatically uses the `ANTHROPIC_API_KEY` environment variable for authentication.

### Available Models
- `claude-sonnet-4-20250514` - Latest Sonnet model
- `claude-opus-4-20250514` - Most capable model for complex tasks
- `claude-haiku-4-20250115` - Fastest model for simple tasks

### Resources
- [Python SDK Documentation](https://github.com/anthropics/anthropic-sdk-python)
- [API Reference](https://docs.anthropic.com/en/api/messages)
- [Model Comparison](https://docs.anthropic.com/en/docs/about-claude/models)

### Basic Message Example

In [4]:
import anthropic

# Initialize client (uses ANTHROPIC_API_KEY from environment)
client = anthropic.Anthropic()

# Basic message creation
message = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    system="You are a helpful AI assistant.",  # System message as top-level parameter
    messages=[
        {"role": "user", "content": "Hello, Claude! Tell me about yourself in 2 sentences."}
    ]
)

print("=== Direct Anthropic Response ===")
print(f"Model: {message.model}")
print(f"Content: {message.content[0].text}")
print(f"Usage: {message.usage.input_tokens} input + {message.usage.output_tokens} output = {message.usage.input_tokens + message.usage.output_tokens} total tokens")

# Full response structure
print("\n=== Full Response Structure ===")
message.model_dump()

=== Direct Anthropic Response ===
Model: claude-sonnet-4-20250514
Content: I'm Claude, an AI assistant created by Anthropic to be helpful, harmless, and honest. I enjoy engaging in conversations on a wide range of topics and helping with various tasks like analysis, writing, math, coding, and creative projects.
Usage: 28 input + 54 output = 82 total tokens

=== Full Response Structure ===


{'id': 'msg_017LYki5Rop3NfJrkNYvWR9D',
 'content': [{'citations': None,
   'text': "I'm Claude, an AI assistant created by Anthropic to be helpful, harmless, and honest. I enjoy engaging in conversations on a wide range of topics and helping with various tasks like analysis, writing, math, coding, and creative projects.",
   'type': 'text'}],
 'model': 'claude-sonnet-4-20250514',
 'role': 'assistant',
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'type': 'message',
 'usage': {'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'input_tokens': 28,
  'output_tokens': 54,
  'server_tool_use': None,
  'service_tier': 'standard'}}

### Advanced Configuration

Claude supports various parameters for fine-tuning responses:

In [7]:
# Advanced configuration example
response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=200,
    temperature=0.7,  # More creative responses
    top_p=0.9,        # Nucleus sampling
    system="You are a helpful AI assistant specializing in creative writing.",
    messages=[
        {
            "role": "user", 
            "content": "Write a creative opening line for a science fiction story."
        }
    ],
)

print("=== Creative Writing Response ===")
print(f"Opening line: {response.content[0].text}")
print(f"Stop reason: {response.stop_reason}")

=== Creative Writing Response ===
Opening line: The last human memory was uploaded at 3:47 AM on a Tuesday, and by Wednesday, the machines had already begun to forget what it felt like to dream.
Stop reason: end_turn


## Function Calling (Tool Use)

Claude supports function calling for interacting with external tools and APIs. Here's how to implement single and multiple tool calls:

### Key Features:
- **Multiple tool calls**: Claude can call multiple tools in a single response
- **Structured inputs**: Tools use JSON schemas for parameter validation
- **Conversation flow**: Tool results can be fed back for continued interaction

### Single Tool Call Example

In [8]:
# Define a weather tool
weather_tool = {
    "name": "get_weather",
    "description": "Get the current weather in a given location",
    "input_schema": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA",
            },
            "units": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "Temperature units"
            }
        },
        "required": ["location"],
    },
}

# Initial request with tool
messages = [{"role": "user", "content": "What's the weather like in San Francisco?"}]

response = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=1024,
    tools=[weather_tool],
    messages=messages,
)

print("=== Tool Call Response ===")
print(f"Content blocks: {len(response.content)}")
for i, content in enumerate(response.content):
    print(f"Block {i}: {content.type}")
    if content.type == "text":
        print(f"  Text: {content.text}")
    elif content.type == "tool_use":
        print(f"  Tool: {content.name}")
        print(f"  ID: {content.id}")
        print(f"  Input: {content.input}")

# Add assistant's response to conversation
messages.append({
    "role": "assistant",
    "content": response.content
})

print(f"\nMessages so far: {len(messages)}")

=== Tool Call Response ===
Content blocks: 2
Block 0: text
  Text: I'll check the current weather in San Francisco for you.
Block 1: tool_use
  Tool: get_weather
  ID: toolu_01Sf5WQ1drYBeFyPLcUssAXv
  Input: {'location': 'San Francisco, CA'}

Messages so far: 2


### Tool Execution and Response

In [9]:
# Simulate tool execution
def execute_weather_tool(location, units="fahrenheit"):
    """Simulate getting weather data"""
    if "san francisco" in location.lower():
        if units == "celsius":
            return "18°C, partly cloudy with light fog"
        else:
            return "65°F, partly cloudy with light fog"
    else:
        return f"Weather data not available for {location}"

# Find tool use in the response
tool_use = None
for content in response.content:
    if content.type == "tool_use":
        tool_use = content
        break

if tool_use:
    # Execute the tool
    location = tool_use.input.get("location")
    units = tool_use.input.get("units", "fahrenheit")
    weather_result = execute_weather_tool(location, units)
    
    print(f"=== Tool Execution ===")
    print(f"Tool: {tool_use.name}")
    print(f"Location: {location}")
    print(f"Result: {weather_result}")
    
    # Send tool result back to Claude
    messages.append({
        "role": "user",
        "content": [
            {
                "type": "tool_result",
                "tool_use_id": tool_use.id,
                "content": weather_result
            }
        ]
    })
    
    # Get final response
    final_response = client.messages.create(
        model="claude-opus-4-20250514",
        max_tokens=1024,
        tools=[weather_tool],
        messages=messages
    )
    
    print(f"\n=== Final Response ===")
    print(f"Claude's response: {final_response.content[0].text}")
else:
    print("No tool use found in response")

=== Tool Execution ===
Tool: get_weather
Location: San Francisco, CA
Result: 65°F, partly cloudy with light fog

=== Final Response ===
Claude's response: The current weather in San Francisco is 65°F (about 18°C), partly cloudy with light fog. This is typical San Francisco weather - mild temperatures with some coastal fog.


### Multiple Tools Example

In [10]:
# Define multiple tools
calculator_tool = {
    "name": "calculator",
    "description": "Perform basic mathematical operations",
    "input_schema": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "The mathematical operation to perform"
            },
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"}
        },
        "required": ["operation", "a", "b"]
    }
}

search_tool = {
    "name": "web_search",
    "description": "Search the web for information",
    "input_schema": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query"
            },
            "max_results": {
                "type": "integer",
                "description": "Maximum number of results",
                "default": 5
            }
        },
        "required": ["query"]
    }
}

# Request that might use multiple tools
multi_tool_response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    tools=[weather_tool, calculator_tool, search_tool],
    messages=[
        {
            "role": "user", 
            "content": "What's 15 * 23? Also, search for information about 'anthropic claude models'"
        }
    ]
)

print("=== Multiple Tools Response ===")
print(f"Total content blocks: {len(multi_tool_response.content)}")

for i, content in enumerate(multi_tool_response.content):
    print(f"\nBlock {i+1}: {content.type}")
    if content.type == "text":
        print(f"  Text: {content.text}")
    elif content.type == "tool_use":
        print(f"  Tool: {content.name}")
        print(f"  ID: {content.id}")
        print(f"  Input: {content.input}")

print(f"\nUsage: {multi_tool_response.usage.input_tokens} + {multi_tool_response.usage.output_tokens} = {multi_tool_response.usage.input_tokens + multi_tool_response.usage.output_tokens} tokens")

=== Multiple Tools Response ===
Total content blocks: 3

Block 1: text
  Text: I'll help you with both requests - calculating 15 * 23 and searching for information about Anthropic Claude models.

Block 2: tool_use
  Tool: calculator
  ID: toolu_014PUoqdUjmh2Vfqd4uzjX9H
  Input: {'operation': 'multiply', 'a': 15, 'b': 23}

Block 3: tool_use
  Tool: web_search
  ID: toolu_01EDhWuJGae4Bk67Ei2arCAg
  Input: {'query': 'anthropic claude models'}

Usage: 646 + 152 = 798 tokens


## Vertex AI Hosting

### Setup and Authentication
Vertex AI requires Google Cloud project configuration and regional settings.

### Key Differences:
- **Project ID**: Must specify your Google Cloud project
- **Region**: Choose deployment region for compliance/latency
- **Model Names**: Slightly different naming convention with `@` version suffix
- **Enterprise Features**: VPC integration, audit logging, and enterprise security

### Available Regions:
- `us-central1` - United States (primary)
- `us-east4` - United States (secondary)
- `europe-west1` - Europe (primary)
- See [Vertex AI Locations](https://cloud.google.com/vertex-ai/docs/general/locations) for complete list

### Resources:
- [Anthropic on Vertex AI Documentation](https://docs.anthropic.com/en/api/claude-on-vertex-ai)
- [Google Cloud Vertex AI](https://cloud.google.com/vertex-ai)
- [Model Garden](https://console.cloud.google.com/vertex-ai/model-garden)

### Basic Vertex AI Example

In [11]:
from anthropic import AnthropicVertex

# Initialize Vertex AI client
project_id = os.getenv("GCP_PROJECT_ID")
region = os.getenv("GCP_LOCATION", "us-central1")  # Default to us-central1

print(f"Connecting to Vertex AI:")
print(f"  Project: {project_id}")
print(f"  Region: {region}")

vertex_client = AnthropicVertex(project_id=project_id, region=region)

# Note: Vertex AI uses @ notation for model versions
vertex_response = vertex_client.messages.create(
    model="claude-sonnet-4@20250514",
    max_tokens=100,
    messages=[
        {
            "role": "user",
            "content": "Hello! Explain the difference between Vertex AI and direct Anthropic hosting.",
        }
    ],
)

print("\n=== Vertex AI Response ===")
print(f"Model: {vertex_response.model}")
print(f"Content: {vertex_response.content[0].text}")
print(f"Usage: {vertex_response.usage.input_tokens} + {vertex_response.usage.output_tokens} = {vertex_response.usage.input_tokens + vertex_response.usage.output_tokens} tokens")

# Full response structure (same as direct Anthropic)
print("\n=== Full Response Structure ===")
vertex_response.model_dump()

Connecting to Vertex AI:
  Project: vectrix-401014
  Region: europe-west1

=== Vertex AI Response ===
Model: claude-sonnet-4-20250514
Content: Here are the key differences between using Vertex AI and direct Anthropic hosting for Claude:

## **Vertex AI (Google Cloud)**

**Pros:**
- **Enterprise integration**: Seamlessly integrates with other Google Cloud services (BigQuery, Cloud Storage, etc.)
- **Unified billing**: Single invoice for all Google Cloud services
- **Enterprise controls**: Advanced IAM, logging, monitoring through Google Cloud Console
- **Compliance**: Leverages Google
Usage: 24 + 100 = 124 tokens

=== Full Response Structure ===


{'id': 'msg_vrtx_01Ei4SdeYxzbZg3S1qgeqKZm',
 'content': [{'citations': None,
   'text': 'Here are the key differences between using Vertex AI and direct Anthropic hosting for Claude:\n\n## **Vertex AI (Google Cloud)**\n\n**Pros:**\n- **Enterprise integration**: Seamlessly integrates with other Google Cloud services (BigQuery, Cloud Storage, etc.)\n- **Unified billing**: Single invoice for all Google Cloud services\n- **Enterprise controls**: Advanced IAM, logging, monitoring through Google Cloud Console\n- **Compliance**: Leverages Google',
   'type': 'text'}],
 'model': 'claude-sonnet-4-20250514',
 'role': 'assistant',
 'stop_reason': 'max_tokens',
 'stop_sequence': None,
 'type': 'message',
 'usage': {'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'input_tokens': 24,
  'output_tokens': 100,
  'server_tool_use': None,
  'service_tier': None}}

### Vertex AI Function Calling

Function calling works identically on Vertex AI as with direct Anthropic hosting:

In [12]:
# Function calling works the same on Vertex AI
vertex_tool_response = vertex_client.messages.create(
    model="claude-sonnet-4@20250514",
    max_tokens=1024,
    tools=[calculator_tool],  # Same tool definition as before
    messages=[
        {"role": "user", "content": "Calculate 42 divided by 7"}
    ]
)

print("=== Vertex AI Tool Call ===")
print(f"Model: {vertex_tool_response.model}")

for content in vertex_tool_response.content:
    if content.type == "text":
        print(f"Text: {content.text}")
    elif content.type == "tool_use":
        print(f"Tool: {content.name}")
        print(f"Operation: {content.input['operation']}")
        print(f"Numbers: {content.input['a']} {content.input['operation']} {content.input['b']}")

print(f"\n✅ Tool calling works identically on Vertex AI!")

=== Vertex AI Tool Call ===
Model: claude-sonnet-4-20250514
Text: I'll calculate 42 divided by 7 for you.
Tool: calculator
Operation: divide
Numbers: 42 divide 7

✅ Tool calling works identically on Vertex AI!


## Comparison: Direct vs Vertex AI

Let's compare the same request across both hosting options:

In [13]:
test_messages = [
    {"role": "user", "content": "Explain quantum computing in exactly 50 words."}
]

print("=== Comparison: Direct vs Vertex AI ===")

# Direct Anthropic
direct_response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=100,
    messages=test_messages
)

print("\n🔵 Direct Anthropic:")
print(f"Model: {direct_response.model}")
print(f"Response: {direct_response.content[0].text}")
print(f"Tokens: {direct_response.usage.input_tokens} + {direct_response.usage.output_tokens}")

# Vertex AI
vertex_response = vertex_client.messages.create(
    model="claude-sonnet-4@20250514",
    max_tokens=100,
    messages=test_messages
)

print("\n🟢 Vertex AI:")
print(f"Model: {vertex_response.model}")
print(f"Response: {vertex_response.content[0].text}")
print(f"Tokens: {vertex_response.usage.input_tokens} + {vertex_response.usage.output_tokens}")

print("\n💡 Both use identical APIs and response formats!")
print("💡 Only differences: authentication method and model naming convention")

=== Comparison: Direct vs Vertex AI ===

🔵 Direct Anthropic:
Model: claude-sonnet-4-20250514
Response: Quantum computing harnesses quantum mechanics principles like superposition and entanglement to process information. Unlike classical bits (0 or 1), quantum bits (qubits) exist in multiple states simultaneously, enabling parallel computations. This allows quantum computers to potentially solve certain complex problems exponentially faster than classical computers, revolutionizing cryptography, optimization, and scientific simulation.
Tokens: 18 + 79

🟢 Vertex AI:
Model: claude-sonnet-4-20250514
Response: Quantum computing harnesses quantum mechanics principles like superposition and entanglement to process information. Unlike classical bits (0 or 1), quantum bits (qubits) exist in multiple states simultaneously, enabling parallel calculations. This allows quantum computers to solve certain complex problems exponentially faster than classical computers, particularly in cryptography, op

## Best Practices

### When to Use Direct Anthropic:
- **Rapid prototyping** and development
- **Latest model access** and features
- **Simple deployment** requirements
- **Global applications** without specific regional needs

### When to Use Vertex AI:
- **Enterprise deployments** with strict security requirements
- **VPC integration** for network isolation
- **Regional compliance** needs (data residency)
- **Unified Google Cloud billing** and management
- **Advanced monitoring** and audit logging requirements

### Error Handling:
```python
try:
    response = client.messages.create(...)  # or vertex_client
except anthropic.APIError as e:
    print(f"API Error: {e}")
except anthropic.RateLimitError as e:
    print(f"Rate limit exceeded: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
```

### Performance Tips:
- **Choose appropriate max_tokens** based on your use case
- **Use streaming** for long responses: `stream=True`
- **Implement retry logic** for production applications
- **Monitor token usage** to optimize costs

## Summary

✅ **Identical APIs**: Both hosting options use the same Anthropic Python SDK  
✅ **Same Features**: Function calling, streaming, and all model capabilities work identically  
✅ **Flexible Deployment**: Choose based on your security, compliance, and infrastructure needs  
✅ **Enterprise Ready**: Vertex AI provides additional enterprise features when needed  

Both direct Anthropic and Vertex AI hosting provide access to the same powerful Claude models with identical APIs, giving you flexibility in how you deploy and scale your applications.